# Face Expression Recognition Demo

This notebook demonstrates how to download the dataset, train the model, and run inference.

In [ ]:
import cv2
import matplotlib.pyplot as plt
import os
import sys
import kagglehub

# Add current directory to path for Colab compatibility
if os.getcwd() not in sys.path:
    sys.path.append(os.getcwd())

from src.face_detector import FaceDetector
from main import load_emotion_model, draw_results
from src.config import DEFAULT_MODEL_PATH
from train import train

# Function to display image in notebook
def show_image(img, title="Image"):
    plt.figure(figsize=(10, 8))
    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    plt.title(title)
    plt.axis('off')
    plt.show()

## 1. Download Dataset
Download the Face Expression Recognition dataset using kagglehub.

In [ ]:
# Download latest version
path = kagglehub.dataset_download("jonathanoheix/face-expression-recognition-dataset")

print("Path to dataset files:", path)

# Check for 'images' subdirectory which is common in this dataset
data_path = path
if os.path.exists(os.path.join(path, 'images')):
    data_path = os.path.join(path, 'images')
    print(f"Adjusted data path to: {data_path}")
    
# Verify train/validation folders exist
if os.path.exists(os.path.join(data_path, 'train')) and os.path.exists(os.path.join(data_path, 'validation')):
    print("Dataset structure verified successfully.")
else:
    print("Warning: 'train' or 'validation' directories not found in expected location. Please check the dataset structure.")

## 2. Train Model
Train the model using the downloaded dataset. This might take a while!

In [ ]:
# Set small epochs for demonstration, increase for better results
EPOCHS = 5
BATCH_SIZE = 32

print("Starting training...")
try:
    train(data_dir=data_path, epochs=EPOCHS, batch_size=BATCH_SIZE)
    print("Training completed!")
except Exception as e:
    print(f"An error occurred during training: {e}")

## 3. Load Model

In [ ]:
model_path = DEFAULT_MODEL_PATH
# Reload the model in case it was just trained
model = load_emotion_model(model_path)

if model:
    print("Model loaded successfully!")
else:
    print("Failed to load model. Please check the path.")

## 4. Run Inference on Test Image

In [ ]:
image_path = "test_face.png"
detector = FaceDetector()

if os.path.exists(image_path):
    frame = cv2.imread(image_path)
    rects = detector.detect_faces(frame)
    frame = draw_results(frame, rects, model)
    show_image(frame, "Emotion Recognition Result")
else:
    print(f"Test image '{image_path}' not found.")